In [1]:
from tqdm import tqdm
import requests
import zipfile
import io
import os
import shutil
import json
import csv
import random
import tarfile

from matplotlib import pyplot as plt

from PIL import Image

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
DATA_DIR= "/home/ubuntu/outdoor"
IMAGES_DIR= DATA_DIR + "/images"
TRAIN_DIR= DATA_DIR + "/train"
VAL_DIR= DATA_DIR + "/val"
DATA_EXT = ".zip"

BASELINE_DIR = "/home/ubuntu/baseline"
BASELINE_IMAGES_DIR = BASELINE_DIR + "/images"

if not os.path.isdir(DATA_DIR): 
    os.mkdir(DATA_DIR)

In [3]:
opts = {
    'image_size':224,
    'rgb_mean':[0.3820, 0.4122, 0.4279],
    'rgb_std':[0.2998, 0.2836, 0.2907],
}

In [4]:
model = torch.load('model.pth')
model.to(device)
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
##From: https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5

test_transforms = transforms.Compose([
        transforms.Resize(opts['image_size']),
        transforms.CenterCrop(opts['image_size']),
        transforms.ToTensor(),
        transforms.Normalize(opts['rgb_mean'], opts['rgb_std'])
    ])

def predict_image(image):
    image_tensor = test_transforms(image).float()
    input = image_tensor.unsqueeze_(0).to(device)
    output = model(input)
    return torch.argmax(output)

def get_random_images(num,directory):
    data = datasets.ImageFolder(directory, transform=test_transforms)
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, 
                   sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = next(dataiter)
    return images, labels

# def get_all_images(directory):
#     data = datasets.ImageFolder(directory, transform=test_transforms)
#     print(len(data))
#     loader = torch.utils.data.DataLoader(
#         dataset=data, 
#         batch_size=1,
#     )
#     dataiter = iter(loader)
#     images, labels = next(dataiter)
#     return images

## Test

In [6]:
classes = ["indoor","outdoor, day","out, night"]

num_images = 100
to_pil = transforms.ToPILImage()
to_tensor = transforms.ToTensor()
images, labels = get_random_images(num_images,TRAIN_DIR)
# fig=plt.figure(figsize=(40,40))
results={0:[0,0],1:[0,0],2:[0,0]}
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    res = int(labels[ii]) == index
    results[int(labels[ii])][int(res)] += 1
#     sub = fig.add_subplot(int(len(images)/10), 10, ii+1)
#     sub.set_title(str(classes[index]) + ":\n" + str(res))
#     plt.axis('off')
#     plt.imshow(image)

for r in results:
    results[r] = results[r][1] / (results[r][1] + results[r][0])
    
print(results)
    
# plt.subplots_adjust(wspace=1,
#                     hspace=.5)
# plt.show()

{0: 0.7333333333333333, 1: 0.6842105263157895, 2: 0.0}


## Label

In [7]:
val_for = ['jpg', 'jpeg', 'png']
path = BASELINE_IMAGES_DIR
images = os.listdir(path)
images = [i for i in images if i.split('.')[-1] in val_for]
labels = []

num_images = len(images)
counts = {0:0,1:0,2:0}

# fig=plt.figure(figsize=(40,40))
for f,file in tqdm(enumerate(images[0:num_images]),total=num_images):
    try:
        image = to_pil(to_tensor(Image.open(path + '/' + file)))
        cls = int(predict_image(image))
        labels.append(cls)
        counts[cls] += 1
    #     sub = fig.add_subplot(int(len(images)/10), 10, f+1)
    #     sub.set_title(str(classes[index]) + ":\n" + str(res))
    #     plt.axis('off')
    #     plt.imshow(image)
    except:
        labels.append('')
        print("Invalid image encountered; skipping")
    
print(counts)
data = {'Image':images[0:num_images], 'Label':labels}
df = pd.DataFrame(data)
df.to_csv(f"{DATA_DIR}/labels-baseline-outdoor_indoor.csv") 
display(df)

# plt.subplots_adjust(wspace=1,
#                     hspace=.5)
# plt.show()

  3%|█▎                                     | 21333/621444 [14:34<6:11:36, 26.92it/s]

Invalid image encountered; skipping


  4%|█▋                                     | 27267/621444 [18:36<6:40:42, 24.71it/s]

Invalid image encountered; skipping


 16%|█████▊                              | 100534/621444 [1:08:12<5:13:55, 27.66it/s]

...

 82%|█████████████████████████████▍      | 507367/621444 [5:49:39<1:13:46, 25.77it/s]

Invalid image encountered; skipping


100%|██████████████████████████████████████| 621444/621444 [7:09:02<00:00, 24.14it/s]


{0: 194111, 1: 347405, 2: 79919}


,Image,Label
0,1127618657.jpg,1
1,4551860347.jpg,0
2,4571931357.jpg,1
3,1433580947.jpg,1
4,1342336304.jpg,0
...,...,...
621439,1348285839.jpg,0
621440,1435948176.jpg,1
621441,4591648556.jpg,0
621442,4568269706.jpg,0
